# Privacy Meter Demo

## Setting up the multi-CPU environment

## Importing

In [1]:
import sys
import math
import time

import numpy as np
import torch
import yaml
from torch.utils.data import Subset

from audit import get_average_audit_results, audit_models, sample_auditing_dataset
from get_signals import get_model_signals
from models.utils import load_models, train_models, split_dataset_for_training
from util import (
    check_configs,
    setup_log,
    initialize_seeds,
    create_directories,
    load_dataset,
)

from trainers.parallel_trainer import parallel_prepare_models
import torch.multiprocessing as mp
if __name__ == '__main__':
    # Required for CUDA multiprocessing
    mp.set_start_method('spawn')

# Enable benchmark mode in cudnn to improve performance when input sizes are consistent
torch.backends.cudnn.benchmark = True

## Load config

In [2]:
configs = "configs/config.yaml"
with open(configs, "rb") as f:
        configs = yaml.load(f, Loader=yaml.Loader)

# Validate configurations
check_configs(configs)

## Setting up

In [3]:
# Validate configurations
check_configs(configs)

# Initialize seeds for reproducibility
initialize_seeds(configs["run"]["random_seed"])

# Create necessary directories
log_dir = configs["run"]["log_dir"]
directories = {
    "log_dir": log_dir,
    "report_dir": f"{log_dir}/report",
    "signal_dir": f"{log_dir}/signals",
    "data_dir": configs["data"]["data_dir"],
}
create_directories(directories)

# Set up logger
logger = setup_log(
    directories["report_dir"], "time_analysis", configs["run"]["time_log"]
)

start_time = time.time()

## Load dataset

In [4]:
baseline_time = time.time()
dataset, population = load_dataset(configs, directories["data_dir"], logger)
logger.info("Loading dataset took %0.5f seconds", time.time() - baseline_time)

2025-02-08 15:34:19,682 INFO     Data loaded from data/cifar10.pkl
2025-02-08 15:34:19,699 INFO     Population data loaded from data/cifar10_population.pkl
2025-02-08 15:34:19,699 INFO     The whole dataset size: 50000
2025-02-08 15:34:19,700 INFO     Loading dataset took 0.09906 seconds


## Load or train models

In [5]:
# Define experiment parameters
num_experiments = configs["run"]["num_experiments"]
num_reference_models = configs["audit"]["num_ref_models"]
num_model_pairs = max(math.ceil(num_experiments / 2.0), num_reference_models + 1)

# Split dataset for training
data_splits, memberships = split_dataset_for_training(
    len(dataset), num_model_pairs
)

In [ ]:
# Now train models in parallel
baseline_time = time.time()
models_list = parallel_prepare_models(
    log_dir, 
    dataset, 
    data_splits,  # Using the generated data_splits
    memberships,  # Using the generated memberships
    configs, 
    logger,
    num_gpus=4
)
logger.info(
    "Model parallel training took %0.1f seconds", 
    time.time() - baseline_time
)

2025-02-08 15:34:26,920 INFO     Training 4 models using 4 GPUs


GPU 0: 0/100 (0.0000|0.0000) | GPU 1: 0/100 (0.0000|0.0000) | GPU 2: 0/100 (0.0000|0.0000) | GPU 3: 0/100 (0.0000|0.0000) | 

## Prepare auditing dataset

In [ ]:
auditing_dataset, auditing_membership = sample_auditing_dataset(
        configs, dataset, logger, memberships
    )

# Also downsample the population set size if specified in the config
population = Subset(
    population,
    np.random.choice(
        len(population),
        configs["audit"].get("population_size", len(population)),
        replace=False,
    ),
)

## Compute signals

In [ ]:
baseline_time = time.time()
signals = get_model_signals(models_list, auditing_dataset, configs, logger)
population_signals = get_model_signals(
        models_list, population, configs, logger, is_population=True
    )
logger.info("Preparing signals took %0.5f seconds", time.time() - baseline_time)

## Audit

In [ ]:
# Perform the privacy audit
baseline_time = time.time()
target_model_indices = list(range(num_experiments))
mia_score_list, membership_list = audit_models(
        f"{directories['report_dir']}/exp",
        target_model_indices,
        signals,
        population_signals,
        auditing_membership,
        num_reference_models,
        logger,
        configs,
    )

if len(target_model_indices) > 1:
    logger.info(
        "Auditing privacy risk took %0.1f seconds", time.time() - baseline_time
    )

# Get average audit results across all experiments
if len(target_model_indices) > 1:
    get_average_audit_results(
        directories["report_dir"], mia_score_list, membership_list, logger
    )

logger.info("Total runtime: %0.5f seconds", time.time() - start_time)